In [2]:
import numpy as np
import matplotlib.pyplot as plt

import requests                                      # reading data
from io import StringIO

from sklearn.datasets import fetch_openml            # common data set access
from sklearn.preprocessing import StandardScaler     # scaling transform
from sklearn.model_selection import train_test_split # validation tools
from sklearn.metrics import zero_one_loss as J01
from sklearn.tree import DecisionTreeClassifier


import sklearn.tree as tree
import math

from pathlib import Path

# Fix the random seed for reproducibility
# !! Important !! : do not change this
seed = 1234
np.random.seed(seed)

DATA_PATH = Path('data/original-dataset')
VOCAB_PATH = DATA_PATH.joinpath('imdb.vocab')
TRAIN_BOW_PATH = DATA_PATH.joinpath('train/labeledBow.feat')
TEST_BOW_PATH = DATA_PATH.joinpath('test/labeledBow.feat')

In [3]:
with open(VOCAB_PATH, 'r', encoding='utf8') as vocab_file:
    VOCAB = np.array(vocab_file.read().split('\n'))
    
print(VOCAB[:10])

['the' 'and' 'a' 'of' 'to' 'is' 'it' 'in' 'i' 'this']


In [9]:
X = []
y = []

with open(TRAIN_BOW_PATH, 'r', encoding='utf8') as train_bow_file:
    n = 1000
    for line in train_bow_file:
        if n == 0:
            break
            
        review = line.split(' ')
        rating = int(review[0])
        y.append(1 if rating > 5 else -1)

        doc_words = [0 for i in range(len(VOCAB))]
        for bow_count in review[1:]:
            vocab_idx, count = bow_count.split(':')
            vocab_idx = int(vocab_idx)
            count = int(count)
            doc_words[vocab_idx] = count
        X.append(doc_words)
        
        n -= 1

X = np.array(X)
y = np.array(y)
print(X.shape)
print(y.shape)
print(X[:10])
print(y[:10])

(1000, 89527)
(1000,)
[[9 1 4 ... 0 0 0]
 [7 4 2 ... 0 0 0]
 [4 4 4 ... 0 0 0]
 ...
 [3 3 6 ... 0 0 0]
 [8 3 2 ... 0 0 0]
 [9 7 8 ... 0 0 0]]
[1 1 1 1 1 1 1 1 1 1]


In [ ]:
X = []
y = []

